In [8]:
import tabula
from tabula import read_pdf

In [7]:
df = tabula.read_pdf("Postcode.pdf")

print (len(df))

33


In [9]:
tabula.convert_into("Postcode.pdf", "Postcode.csv", output_format="csv", pages='all')

In [6]:
import pandas as pd

df = pd.read_csv('Postcode2.csv',  encoding="ISO-8859-1")
df.head(5)

68967


In [9]:
multi_tables = tabula.read_pdf("Postcode.pdf", pages="all", multiple_tables=True, lattice=True)
print (len(multi_tables))

1789


In [10]:
for table in multi_tables:
    print (table.head(10))

           0         1                 2         3              4  \
0     REGION  POSTCODE              WARD  POSTCODE  OLD\rPOSTCODE   
1  ILALA CBD        11          KIVUKONI     11101            NaN   
2        NaN       NaN               NaN       NaN            NaN   
3        NaN       NaN  UPANGA MASHARIKI     11102            NaN   
4        NaN       NaN               NaN       NaN            NaN   
5        NaN       NaN  UPANGA MAGHARIBI     11103            NaN   
6        NaN       NaN               NaN       NaN            NaN   
7        NaN       NaN               NaN       NaN            NaN   
8        NaN       NaN            KISUTU     11104            NaN   
9        NaN       NaN               NaN       NaN            NaN   

              5          6  
0  MTAA/VILLAGE  KITONGOJI  
1      Kivukoni        NaN  
2      Sea View        NaN  
3       Kitonga        NaN  
4      Kibasila        NaN  
5          Fire        NaN  
6      Charambe        NaN  
7       

9 NaN NaN NaN NaN        NaN    NaN NaN  Msambweni "A" NaN
    0   1    2    3          4      5   6                7   8
0 NaN NaN  NaN  NaN        NaN    NaN NaN   Duga Viwanjani NaN
1 NaN NaN  NaN  NaN        NaN    NaN NaN        Duga Mpya NaN
2 NaN NaN  NaN  NaN        NaN    NaN NaN  Duga Barabarani NaN
3 NaN NaN  NaN  NaN    Magaoni  21115 NaN       Miembeni A NaN
4 NaN NaN  NaN  NaN        NaN    NaN NaN       Miembeni B NaN
5 NaN NaN  NaN  NaN        NaN    NaN NaN        Magaoni A NaN
6 NaN NaN  NaN  NaN        NaN    NaN NaN        Magaoni B NaN
7 NaN NaN  NaN  NaN        NaN    NaN NaN          Swahili NaN
8 NaN NaN  NaN  NaN        NaN    NaN NaN            Kiboi NaN
9 NaN NaN  NaN  NaN  Mnyanjani  21116 NaN        Mnyanjani NaN
    0   1    2    3            4      5   6            7    8
0 NaN NaN  NaN  NaN          NaN    NaN NaN    Pande 'A'  NaN
1 NaN NaN  NaN  NaN          NaN    NaN NaN     Mabayani  NaN
2 NaN NaN  NaN  NaN          NaN    NaN NaN     Kivuleni  NaN


    0   1   2   3         4      5   6         7                 8
0 NaN NaN NaN NaN       NaN    NaN NaN       NaN           Shuleni
1 NaN NaN NaN NaN       NaN    NaN NaN       NaN           Majengo
2 NaN NaN NaN NaN  Mapatano  21506 NaN  Mapatano  Mapatano shuleni
3 NaN NaN NaN NaN       NaN    NaN NaN       NaN      Mapatano CCM
4 NaN NaN NaN NaN       NaN    NaN NaN       NaN         Kwamfaume
5 NaN NaN NaN NaN       NaN    NaN NaN       NaN       Kwamachemko
6 NaN NaN NaN NaN       NaN    NaN NaN       NaN           Kibaoni
7 NaN NaN NaN NaN       NaN    NaN NaN       NaN      Legezamwendo
8 NaN NaN NaN NaN       NaN    NaN NaN   Mtakuja       Mtakuja ‘A’
9 NaN NaN NaN NaN       NaN    NaN NaN       NaN       Mtakuja ‘B’
    0   1   2   3    4    5   6                 7                 8
0 NaN NaN NaN NaN  NaN  NaN NaN               NaN  Lanzoni Darajani
1 NaN NaN NaN NaN  NaN  NaN NaN               NaN         Mavengero
2 NaN NaN NaN NaN  NaN  NaN NaN         Mchangani      Kich

    0   1   2   3    4    5   6                7           8
0 NaN NaN NaN NaN  NaN  NaN NaN              NaN     Masanga
1 NaN NaN NaN NaN  NaN  NaN NaN              NaN       Kumba
2 NaN NaN NaN NaN  NaN  NaN NaN              NaN     Kihingo
3 NaN NaN NaN NaN  NaN  NaN NaN  Foroforo Kiuzai        Tewe
4 NaN NaN NaN NaN  NaN  NaN NaN              NaN      Maweni
5 NaN NaN NaN NaN  NaN  NaN NaN              NaN    Foroforo
6 NaN NaN NaN NaN  NaN  NaN NaN              NaN  Kiuzai 'A'
7 NaN NaN NaN NaN  NaN  NaN NaN              NaN  Kiuzai 'B'
8 NaN NaN NaN NaN  NaN  NaN NaN         Kwenkeyu    Kwenkeyu
9 NaN NaN NaN NaN  NaN  NaN NaN              NaN     Magunga
    0   1    2    3    4    5   6            7              8
0 NaN NaN  NaN  NaN  NaN  NaN NaN          NaN      Kwambondo
1 NaN NaN  NaN  NaN  NaN  NaN NaN          NaN  Sisi Kwa Sisi
2 NaN NaN  NaN  NaN  NaN  NaN NaN          NaN         Relini
3 NaN NaN  NaN  NaN  NaN  NaN NaN          NaN       Zahanati
4 NaN NaN  NaN  NaN

    0   1   2   3       4      5   6           7           8
0 NaN NaN NaN NaN     NaN    NaN NaN         NaN     Majengo
1 NaN NaN NaN NaN     NaN    NaN NaN    Kwamzuza    Bagamoyo
2 NaN NaN NaN NaN     NaN    NaN NaN         NaN     Mzizima
3 NaN NaN NaN NaN     NaN    NaN NaN         NaN   Kwemikuyu
4 NaN NaN NaN NaN  Mponde  21728 NaN  Kweminyasa  Kweminyasa
5 NaN NaN NaN NaN     NaN    NaN NaN         NaN     Mabanda
6 NaN NaN NaN NaN     NaN    NaN NaN         NaN     Mbughui
7 NaN NaN NaN NaN     NaN    NaN NaN         NaN     Magundi
8 NaN NaN NaN NaN     NaN    NaN NaN         NaN      Kilole
9 NaN NaN NaN NaN     NaN    NaN NaN         NaN        Zeba
    0   1   2   3   4   5   6     7          8
0 NaN NaN NaN NaN NaN NaN NaN   NaN      Bwiko
1 NaN NaN NaN NaN NaN NaN NaN   NaN    Lwandai
2 NaN NaN NaN NaN NaN NaN NaN  Wena   Mkambini
3 NaN NaN NaN NaN NaN NaN NaN   NaN     Mbazii
4 NaN NaN NaN NaN NaN NaN NaN   NaN      Ngina
5 NaN NaN NaN NaN NaN NaN NaN   NaN  Wena Kaya


...

    0   1   2   3      4      5   6                 7   8
0 NaN NaN NaN NaN    NaN    NaN NaN           Nughutu NaN
1 NaN NaN NaN NaN    NaN    NaN NaN          Chalumbi NaN
2 NaN NaN NaN NaN  Bigwa  67111 NaN  Bigwa Barabarani NaN
3 NaN NaN NaN NaN    NaN    NaN NaN            Stendi NaN
4 NaN NaN NaN NaN    NaN    NaN NaN            Lukuyu NaN
5 NaN NaN NaN NaN    NaN    NaN NaN         Kindimbwa NaN
6 NaN NaN NaN NaN    NaN    NaN NaN       Misongeni A NaN
7 NaN NaN NaN NaN    NaN    NaN NaN          Zahanati NaN
8 NaN NaN NaN NaN    NaN    NaN NaN       Misongeni B NaN
9 NaN NaN NaN NaN    NaN    NaN NaN         Transfoma NaN
    0   1   2   3        4      5   6                  7   8
0 NaN NaN NaN NaN      NaN    NaN NaN          Riverside NaN
1 NaN NaN NaN NaN      NaN    NaN NaN        Mwembesongo NaN
2 NaN NaN NaN NaN      NaN    NaN NaN           Makaburi NaN
3 NaN NaN NaN NaN      NaN    NaN NaN        Ndege wengi NaN
4 NaN NaN NaN NaN  Kihonda  67114 NaN        Kilimanjaro 

    0   1   2   3    4    5   6       7              8
0 NaN NaN NaN NaN  NaN  NaN NaN     NaN        Bombani
1 NaN NaN NaN NaN  NaN  NaN NaN     NaN     Barabarani
2 NaN NaN NaN NaN  NaN  NaN NaN     NaN        Mgudeni
3 NaN NaN NaN NaN  NaN  NaN NaN     NaN      Jitegemee
4 NaN NaN NaN NaN  NaN  NaN NaN     NaN          Umoja
5 NaN NaN NaN NaN  NaN  NaN NaN     NaN  Kibwengongolo
6 NaN NaN NaN NaN  NaN  NaN NaN     NaN           Kota
7 NaN NaN NaN NaN  NaN  NaN NaN  Mbwade        Duthumi
8 NaN NaN NaN NaN  NaN  NaN NaN     NaN     Chang'ombe
9 NaN NaN NaN NaN  NaN  NaN NaN     NaN     Toangoma A
    0   1   2   3    4    5   6         7          8
0 NaN NaN NaN NaN  NaN  NaN NaN  Temekero      Idike
1 NaN NaN NaN NaN  NaN  NaN NaN       NaN       Fulu
2 NaN NaN NaN NaN  NaN  NaN NaN       NaN  Nyamisaga
3 NaN NaN NaN NaN  NaN  NaN NaN       NaN    Kibundi
4 NaN NaN NaN NaN  NaN  NaN NaN   Lukange    Tandali
5 NaN NaN NaN NaN  NaN  NaN NaN       NaN     Lowero
6 NaN NaN NaN NaN  NaN  

    0    1   2   3    4    5   6         7         8
0 NaN  NaN NaN NaN  NaN  NaN NaN   Ndugutu  Ndungutu
1 NaN  NaN NaN NaN  NaN  NaN NaN       NaN    Kitiba
2 NaN  NaN NaN NaN  NaN  NaN NaN       NaN     Lundi
3 NaN  NaN NaN NaN  NaN  NaN NaN       NaN   Mahanga
4 NaN  NaN NaN NaN  NaN  NaN NaN       NaN      Dosa
5 NaN  NaN NaN NaN  NaN  NaN NaN  Kibagala  Kibagala
6 NaN  NaN NaN NaN  NaN  NaN NaN       NaN    Kinavu
7 NaN  NaN NaN NaN  NaN  NaN NaN       NaN      Silu
8 NaN  NaN NaN NaN  NaN  NaN NaN       NaN  Chembela
9 NaN  NaN NaN NaN  NaN  NaN NaN       NaN    Lumbwe
    0   1   2   3        4      5   6          7          8
0 NaN NaN NaN NaN      NaN    NaN NaN        NaN     Vigugu
1 NaN NaN NaN NaN      NaN    NaN NaN        NaN     mikuyu
2 NaN NaN NaN NaN      NaN    NaN NaN        NaN     Mganga
3 NaN NaN NaN NaN      NaN    NaN NaN        NaN     Lukumi
4 NaN NaN NaN NaN  Lubungo  67326 NaN  Lubungo B    Lubungo
5 NaN NaN NaN NaN      NaN    NaN NaN        NaN  Chembwe

    0    1   2   3    4    5   6       7            8
0 NaN  NaN NaN NaN  NaN  NaN NaN     NaN    Kibungumo
1 NaN  NaN NaN NaN  NaN  NaN NaN     NaN       Magaga
2 NaN  NaN NaN NaN  NaN  NaN NaN     NaN        Mrowa
3 NaN  NaN NaN NaN  NaN  NaN NaN     NaN        Mjini
4 NaN  NaN NaN NaN  NaN  NaN NaN     NaN  Singula "A"
5 NaN  NaN NaN NaN  NaN  NaN NaN     NaN  Singula "B"
6 NaN  NaN NaN NaN  NaN  NaN NaN     NaN    Lengewaha
7 NaN  NaN NaN NaN  NaN  NaN NaN  Mlunga   Mnazi Moja
8 NaN  NaN NaN NaN  NaN  NaN NaN     NaN      Majengo
9 NaN  NaN NaN NaN  NaN  NaN NaN     NaN     Mabatini
     0   1   2    3   4   5   6             7             8
0  NaN NaN NaN  NaN NaN NaN NaN           NaN       Gulioni
1  NaN NaN NaN  NaN NaN NaN NaN   Tindiga 'B'       Shuleni
2  NaN NaN NaN  NaN NaN NaN NaN           NaN      Chamwino
3  NaN NaN NaN  NaN NaN NaN NaN           NaN        Mbwade
4  NaN NaN NaN  NaN NaN NaN NaN           NaN        Kiegea
5  NaN NaN NaN  NaN NaN NaN NaN           NaN 

9 NaN NaN  NaN  NaN  NaN  NaN NaN  Igumbiro  Igumbiro Kati
    0   1   2   3        4      5   6        7            8
0 NaN NaN NaN NaN      NaN    NaN NaN  Kisitwi      Shuleni
1 NaN NaN NaN NaN      NaN    NaN NaN      NaN     Manyemba
2 NaN NaN NaN NaN      NaN    NaN NaN      NaN      Mbuyuni
3 NaN NaN NaN NaN      NaN    NaN NaN      NaN  Ng'holongwa
4 NaN NaN NaN NaN      NaN    NaN NaN   Muheza       Muheza
5 NaN NaN NaN NaN      NaN    NaN NaN      NaN        Ititu
6 NaN NaN NaN NaN      NaN    NaN NaN      NaN     Mabatini
7 NaN NaN NaN NaN      NaN    NaN NaN      NaN     Mkombeji
8 NaN NaN NaN NaN  Mandege  67703 NaN  Njungwa      Njungwa
9 NaN NaN NaN NaN      NaN    NaN NaN      NaN     Midenghe
    0   1   2   3    4    5   6         7                 8
0 NaN NaN NaN NaN  NaN  NaN NaN       NaN        Chinangali
1 NaN NaN NaN NaN  NaN  NaN NaN    Mkobwe    Mkobwe Misheni
2 NaN NaN NaN NaN  NaN  NaN NaN       NaN  Mkobwe Njiapanda
3 NaN NaN NaN NaN  NaN  NaN NaN       NaN

In [75]:
print (multi_tables[0].columns.values)

[0 1 2 3 4 5 6]


##### Attempt to combine the different pages into regional tables for future combination 

In [76]:
import pandas as pd

def combine_tables(multi_tables):
    df_list = []
    
    for table in multi_tables:
        if table.loc[0,0] == 'REGION':
            try:
                df_list.append(temp_df)
            except UnboundLocalError:
                pass
            
            temp_df = table
        else:
            temp_df = pd.concat([temp_df, table])
    return df_list
            

In [77]:
regions = combine_tables(multi_tables)

In [78]:
for region in regions:
    print (region.iloc[1,0])
    

ILALA CBD
TANGA
ARUSHA
KILIMANJARO
MANYARA
GEITA
MARA
MWANZA
KAGERA
SHINYANGA
SIMIYU
DODOMA
SINGIDA
TABORA
KIGOMA
KATAVI
IRINGA
MBEYA
RUKWA
RUVUMA
NJOMBE
PWANI
MTWARA
LINDI


In [88]:
def create_columns(df):
    #extract the first row of the table and replace
    #the column headers with the first row.
    columns = df.columns.values
    #new_cols = df.iloc[0, columns].values
    new_cols = df.iloc[0]
    print ('Columns: {}, New columns: {}'.format(columns, new_cols))
    
    rename_dict = {}
    for column in columns:
        rename_dict[column] = new_cols[column]
    
    df.rename(columns=rename_dict, inplace=True)
    return df
    
    
def clean_table(df):
    #perform forward fill for each column in the tables
    
    for column in df:
        df[column] = df[column].fillna(method='ffill')
    return df

In [80]:
new_regions = []
for region in regions:
    temp = create_columns(region)
    print (temp.head(5))
    new_regions.append(temp)
    

Columns: [0 1 2 3 4 5 6], New columns: 0           REGION
1         POSTCODE
2             WARD
3         POSTCODE
4    OLD\rPOSTCODE
5     MTAA/VILLAGE
6        KITONGOJI
Name: 0, dtype: object
      REGION  POSTCODE              WARD  POSTCODE  OLD\rPOSTCODE  \
0     REGION  POSTCODE              WARD  POSTCODE  OLD\rPOSTCODE   
1  ILALA CBD        11          KIVUKONI     11101            NaN   
2        NaN       NaN               NaN       NaN            NaN   
3        NaN       NaN  UPANGA MASHARIKI     11102            NaN   
4        NaN       NaN               NaN       NaN            NaN   

   MTAA/VILLAGE  KITONGOJI  
0  MTAA/VILLAGE  KITONGOJI  
1      Kivukoni        NaN  
2      Sea View        NaN  
3       Kitonga        NaN  
4      Kibasila        NaN  
Columns: [0 1 2 3 4 5 6 7 8], New columns: 0           REGION
1         POSTCODE
2         DISTRICT
3         POSTCODE
4             WARD
5         POSTCODE
6    OLD\rPOSTCODE
7     MTAA/VILLAGE
8        KITONGOJI
Na

   REGION  POSTCODE    DISTRICT  POSTCODE          WARD  POSTCODE  \
0  REGION  POSTCODE    DISTRICT  POSTCODE          WARD  POSTCODE   
1  NJOMBE        59  NJOMBE CBD       591  Njombe Mjini     59101   
2     NaN       NaN         NaN       NaN           NaN       NaN   
3     NaN       NaN         NaN       NaN           NaN       NaN   
4     NaN       NaN         NaN       NaN           NaN       NaN   

   OLD\rPOSTCODE  MTAA/VILLAGE  KITONGOJI  
0  OLD\rPOSTCODE  MTAA/VILLAGE  KITONGOJI  
1            NaN       Kwivaha        NaN  
2            NaN    Posta kati        NaN  
3            NaN           NHC        NaN  
4            NaN      Mgendela        NaN  
Columns: [0 1 2 3 4 5 6 7 8], New columns: 0           REGION
1         POSTCODE
2         DISTRICT
3         POSTCODE
4             WARD
5         POSTCODE
6    OLD\rPOSTCODE
7     MTAA/VILLAGE
8        KITONGOJI
Name: 0, dtype: object
   REGION  POSTCODE    DISTRICT  POSTCODE   WARD  POSTCODE  OLD\rPOSTCODE  \
0  REGI

In [81]:
new_regions[0].head(5)

,REGION,POSTCODE,WARD,POSTCODE,OLD POSTCODE,MTAA/VILLAGE,KITONGOJI
0,REGION,POSTCODE,WARD,POSTCODE,OLD\rPOSTCODE,MTAA/VILLAGE,KITONGOJI
1,ILALA CBD,11,KIVUKONI,11101,NaN,Kivukoni,NaN
2,NaN,NaN,NaN,NaN,NaN,Sea View,NaN
3,NaN,NaN,UPANGA MASHARIKI,11102,NaN,Kitonga,NaN
4,NaN,NaN,NaN,NaN,NaN,Kibasila,NaN


In [89]:
final_regions = []
for region in new_regions:
    temp = clean_table(region)
    final_regions.append(temp)

In [91]:
final_regions[1].head(100)

,REGION,POSTCODE,DISTRICT,POSTCODE,WARD,POSTCODE,OLD POSTCODE,MTAA/VILLAGE,KITONGOJI
0,REGION,POSTCODE,DISTRICT,POSTCODE,WARD,POSTCODE,OLD\rPOSTCODE,MTAA/VILLAGE,KITONGOJI
1,TANGA,21,TANGA CBD,211,Central,21101,OLD\rPOSTCODE,Raskazone,KITONGOJI
2,TANGA,21,TANGA CBD,211,Central,21101,OLD\rPOSTCODE,Bombo Area,KITONGOJI
3,TANGA,21,TANGA CBD,211,Central,21101,OLD\rPOSTCODE,Ambaoni Road,KITONGOJI
4,TANGA,21,TANGA CBD,211,Central,21101,OLD\rPOSTCODE,Darajani,KITONGOJI
5,TANGA,21,TANGA CBD,211,Central,21101,OLD\rPOSTCODE,Sakarani,KITONGOJI
6,TANGA,21,TANGA CBD,211,Central,21101,OLD\rPOSTCODE,Posta,KITONGOJI
7,TANGA,21,TANGA CBD,211,Central,21101,OLD\rPOSTCODE,Central,KITONGOJI
8,TANGA,21,TANGA CBD,211,Central,21101,OLD\rPOSTCODE,New Hotel,KITONGOJI
9,TANGA,21,TANGA CBD,211,Central,21101,OLD\rPOSTCODE,Halmashauri ya Jiji,KITONGOJI
